Name: Oğuz Kaan Yüksel

I hereby declare that I observed the honour code of the university when preparing the homework.

## Pr?gr?mm?ng?H?m?w?rk

In this exercise we model a string of text using a Markov(1) model. For simplicity we only consider letters 'a-z'. Capital letters 'A-Z' are mapped to the corresponding ones. All remaining letters, symbols, numbers, including spaces, are denoted by '.'.


We have a probability table $T$ where $T_{i,j} = p(x_t = j | x_{t-1} = i)$  transition model of letters in English text for $t=1,2 \dots N$. Assume that the initial letter in a string is always a space denoted as $x_0 = \text{'.'}$. Such a model where the probability table is always the same is sometimes called a stationary model.

1. For a given $N$, write a program to sample random strings with letters $x_1, x_2, \dots, x_N$ from $p(x_{1:N}|x_0)$
1. Now suppose you are given strings with missing letters, where each missing letter is denoted by a question mark (or underscore, as below). Implement a method, that samples missing letters conditioned on observed ones, i.e., samples from $p(x_{-\alpha}|x_{\alpha})$ where $\alpha$ denotes indices of observed letters. For example, if the input is 't??.', we have $N=4$ and
$x_1 = \text{'t'}$ and $x_4 = \text{'.'}$, $\alpha=\{1,4\}$ and $-\alpha=\{2,3\}$. Your program may possibly generate the strings 'the.', 'twi.', 'tee.', etc. Hint: make sure to make use all data given and sample from the correct distribution. Implement the method and print the results for the test strings below. 
1. Describe a method for filling in the gaps by estimating the most likely letter for each position. Hint: you need to compute
$$
x_{-\alpha}^* = \arg\max_{x_{-\alpha}} p(x_{-\alpha}|x_{\alpha})
$$
Implement the method and print the results for the following test strings along with the log-probability  $\log p(x_{-\alpha}^*,x_{\alpha})$.
1. Discuss how you can improve the model to get better estimations.

In [198]:
test_strings = ['th__br__n.f_x.', '_u_st__n_.to_be._nsw_r__','i__at_._a_h_n_._e_r_i_g','q___t.___z._____t.__.___.__.']

Hint: The code below loads a table of transition probabilities for English text.

In [199]:
import csv
from IPython.display import display, Latex

alphabet = [chr(i+ord('a')) for i in range(26)]
alphabet.append('.')
letter2idx = {c:i for i,c in enumerate(alphabet)}

T = []
with open('transitions.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        T.append(row)

print('Example')
## p(x_t = 'u' | x_{t-1} = 'q')
display(Latex(r"$p(x_t = \text{'u'} | x_{t-1} = \text{'q'})$"))
print(T[letter2idx['q']][letter2idx['u']])
display(Latex(r"$p(x_t | x_{t-1} = \text{'a'})$"))
for c,p in zip(alphabet,T[letter2idx['a']]):
    print(c,p)

Example


<IPython.core.display.Latex object>

0.9949749


<IPython.core.display.Latex object>

a 0.0002835
b 0.0228302
c 0.0369041
d 0.0426290
e 0.0012216
f 0.0075739
g 0.0171385
h 0.0014659
i 0.0372661
j 0.0002353
k 0.0110124
l 0.0778259
m 0.0260757
n 0.2145354
o 0.0005459
p 0.0195213
q 0.0001749
r 0.1104770
s 0.0934290
t 0.1317960
u 0.0098029
v 0.0306574
w 0.0088799
x 0.0009562
y 0.0233701
z 0.0018701
. 0.0715219


Let's make some observations based on the Markov(1) model. 
1. Note that $T$ table is independent of $t = 1,2,3,...N$.
2. Note the conditional independence i.e $x_n \amalg x_m | x_k$ where $n < k < m$. 

Therefore we can make the following calculations:
1. Conditional table $p(x_{t+k} = j | x_t = i)$ is not dependent on $t$ and this table represented as $table[i][j][k]$.
2. Probability of a string $x_1x_2..x_N$ is given by: $p(x_1, x_2, .., x_N) = p(x_1 | x_0)p(x_2 | x_1) .. p(x_N | x_{N-1})$
3. By Bayes theorem and conditional independence on Markov(1), we have: $p(x_k | x_n, x_m) = p(x_m | x_k) * p(x_k | x_n) / p(x_m | x_n)$ where $m < k < n$.

In [204]:
import random

N = 50 # maximum string length supported (used in dynamic calculations)

table = [[[-1 for k in range(N)] for j in range(len(alphabet))] for i in range(len(alphabet))]
def cond(a, b, k=1): # calculate b | a where dist(a, b) = k i.e b = x_k+n, a = x_n
    if table[a][b][k] != -1:
        return table[a][b][k]
    if k == 1:
        table[a][b][k] = float(T[a][b])
    else:
        table[a][b][k] = 0
        for c in alphabet:
            table[a][b][k] = table[a][b][k] + cond(letter2idx[c], b, k-1) * cond(a, letter2idx[c], 1)
    return table[a][b][k]

def prob(s): # calculate probability of string
    p = cond(letter2idx['.'], letter2idx[s[0]]) # prob of first element
    for i in range(len(s)-1):
        p = p * cond(letter2idx[s[i]], letter2idx[s[i+1]])
    return p

def cond2(a, b, c, k, l): # calculate b | a, c where dist(a, b) = k and dist(b, c) = l i.e c = x_k+l+n, b = x_k+n, a = x_n
    return cond(b, c, l) * cond(a, b, k) / cond(a, c, k+l)

### 1. Sample random string with given length N.

In [206]:
def gen(length):
    s = '.' # initial string
    for i in range(length):
        p = random.uniform(0,1) # take random value in (0,1)
        for c in alphabet:
            p = p - cond(letter2idx[s[-1]], letter2idx[c]) # if p falls in this letter in CDF
            if p <= 0:
                s += c # append the char
                break
    return s[1:]

s = gen(5)
print(s)
print(prob(s))

iot.l
9.90063340453552e-07


### 2. Fill missing letters in given string.

In [217]:
def fill(s):
    s = '.' + s # append initial x_0
    left = 0 # last known left char
    right = 0 # last known right char

    for i in range(len(s)): # start filling
        print(i)
        print(s)
        if s[i] == '?' or s[i] == '_': # if missing char
            if right < i: # if right char is outdated
                right = i + 1
                while right < len(s): # updated right char
                    if s[right] != '?' and s[right] != '_':
                        break # located a known char
                    right = right + 1

            p = random.uniform(0,1) # take random value in (0,1)
            if right == len(s): # if right char is out of bounds, condition only on left
                for c in alphabet:
                    p = p - cond(letter2idx[s[left]], letter2idx[c], i-left)
                    if p <= 0:
                        s = s[:i] + c + s[i+1:] # assign new char
                        break
            else: # double conditional
                for c in alphabet:
                    p = p - cond2(letter2idx[s[left]], letter2idx[c], letter2idx[s[right]], i-left, right-i)
                    if p <= 0:
                        s = s[:i] + c + s[i+1:] # assign new char
                        break

            left = i # update left char
        else:
            left = i # update left char
    return s[1:]

for s in test_strings:
    print(fill(s))

0
.th__br__n.f_x.
1
.th__br__n.f_x.
2
.th__br__n.f_x.
3
.th__br__n.f_x.
4
.the_br__n.f_x.
5
.the.br__n.f_x.
6
.the.br__n.f_x.
7
.the.br__n.f_x.
8
.the.br._n.f_x.
9
.the.br.an.f_x.
10
.the.br.an.f_x.
11
.the.br.an.f_x.
12
.the.br.an.f_x.
13
.the.br.an.fex.
14
.the.br.an.fex.
the.br.an.fex.
0
._u_st__n_.to_be._nsw_r__
1
._u_st__n_.to_be._nsw_r__
2
.au_st__n_.to_be._nsw_r__
3
.au_st__n_.to_be._nsw_r__
4
.aurst__n_.to_be._nsw_r__
5
.aurst__n_.to_be._nsw_r__
6
.aurst__n_.to_be._nsw_r__
7
.aurst._n_.to_be._nsw_r__
8
.aurst.in_.to_be._nsw_r__
9
.aurst.in_.to_be._nsw_r__
10
.aurst.in..to_be._nsw_r__
11
.aurst.in..to_be._nsw_r__
12
.aurst.in..to_be._nsw_r__
13
.aurst.in..to_be._nsw_r__
14
.aurst.in..to.be._nsw_r__
15
.aurst.in..to.be._nsw_r__
16
.aurst.in..to.be._nsw_r__
17
.aurst.in..to.be._nsw_r__
18
.aurst.in..to.be.answ_r__
19
.aurst.in..to.be.answ_r__
20
.aurst.in..to.be.answ_r__
21
.aurst.in..to.be.answ_r__
22
.aurst.in..to.be.answor__
23
.aurst.in..to.be.answor__
24
.aurst.in..to.be.answ

### 3. Most likely letter for each position. (BROKEN)

In [203]:
def fill_map(s):
    s = '.' + s # append initial x_0
    left = 0 # last known left char
    right = 0 # last known right char

    pp = 1
    for i in range(len(s)): # start filling
        if s[i] == '?' or s[i] == '_': # if missing char
            if right < i: # if right char is outdated
                right = i + 1
                while right < len(s): # updated right char
                    if s[right] != '?' and s[right] != '_':
                        break # located a known char
                    right = right + 1

            p = 0
            ch = '.'
            if right == len(s): # if right char is out of bounds, condition only on left
                for c in alphabet:
                    if p < cond(letter2idx[s[left]], letter2idx[c], i-left): # determine max
                        p = cond(letter2idx[s[left]], letter2idx[c], i-left)
                        ch = c # set map char
                        break
            else: # double conditional
                for c in alphabet:
                    if p < cond2(letter2idx[s[left]], letter2idx[c], letter2idx[s[right]], i-left, right-i):
                        p = cond2(letter2idx[s[left]], letter2idx[c], letter2idx[s[right]], i-left, right-i)
                        ch = c # set map char
                        break

            pp = pp * p
            s = s[:i] + ch + s[i+1:] # assign new char
        else:
            left = i # update left char (only updated in knowns)
    return (s[1:], pp)

for s in test_strings:
    print(fill_map(s))

('thaabraan.fax.', 1.9786077107529037e-06)
('auastaana.toabe.answaraa', 3.5823912517589296e-13)
('iaaata.aaahana.aearaiag', 4.726260775675123e-26)
('qraat.aaaz.aaaaat.aa.aaa.aa.', 4.077396454147246e-24)


### 4. Possible Model Improvements

- One can impose further relationship between characters with distance 2,3,4 and so on on the Markov model(1) to capture the language structure more tightly.
- One can add hidden states between characters and learn these hidden states from the observed states to detect further semantics such as role of characters etc.
- One can also add hidden states for groups of characters to further understand semantic of patterns of consequent characters.